In [2]:
from docx import Document
from docx.shared import Inches
import pandas as pd
import matplotlib.pyplot as plt
import os

# Load Excel file
#file_path = "724000005- Hafiza Trading 4557562_07_Day Arc.xls"
df = pd.read_csv("724000005- Hafiza Trading 4557562_07_Day Arc.csv")

# Parse and prepare the data
df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d,%H:%M:%S', errors='coerce')
df.dropna(subset=['Time'], inplace=True)
df.set_index('Time', inplace=True)

df.rename(columns={
    'Vb [m3]': 'Uncorrected Volume',
    'Vbt [m3]': 'Total Uncorrected Volume',
    'Vm [m3]': 'Corrected Volume',
    'Vmt [m3]': 'Total Corrected Volume',
    'Diffrence': 'Hourly Usage',
    'p.MP  [ bar]': 'Pressure (bar)',
    'T.MP [{C]': 'Temperature (C)'
}, inplace=True)

# Convert pressure to psig
df['Pressure (psig)'] = df['Pressure (bar)'] * 14.5038

# Create Word document
doc = Document()
doc.add_heading('Daily Gas Usage, Pressure & Temperature - Hafiza Traders Pvt. Ltd', level=1)

# Output directory
plot_dir = "PlotsData"
os.makedirs(plot_dir, exist_ok=True)

# Plot per day
for date in df.index.normalize().unique():
    daily = df.loc[date.strftime('%Y-%m-%d')]

    fig, ax1 = plt.subplots(figsize=(12, 4))
    ax1.set_title(f"Daily Gas Usage, Pressure & Temperature - {date.strftime('%Y-%m-%d')}")
    ax1.set_xlabel("Time")

    ax1.plot(daily.index, daily['Hourly Usage'], color='blue', label='Hourly Usage (m³)')
    ax1.set_ylabel("Usage (m³)", color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')

    # Pressure
    ax2 = ax1.twinx()
    ax2.plot(daily.index, daily['Pressure (psig)'], color='red', label='Pressure (psig)')
    ax2.set_ylabel("Pressure (psig)", color='red')
    ax2.tick_params(axis='y', labelcolor='red')

    # Temperature (3rd axis using offset)
    ax3 = ax1.twinx()
    ax3.spines["right"].set_position(("outward", 60))
    ax3.plot(daily.index, daily['Temperature (C)'], color='green', label='Temperature (°C)')
    ax3.set_ylabel("Temperature (°C)", color='green')
    ax3.tick_params(axis='y', labelcolor='green')

    # Combine legends
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    lines3, labels3 = ax3.get_legend_handles_labels()
    ax1.legend(lines + lines2 + lines3, labels + labels2 + labels3, loc='upper left', bbox_to_anchor=(1, 1))

    plt.tight_layout()
    plot_path = os.path.join(plot_dir, f"{date.strftime('%Y-%m-%d')}.png")
    plt.savefig(plot_path)
    plt.close()

    doc.add_heading(date.strftime('Daily Summary - %Y-%m-%d'), level=2)
    doc.add_picture(plot_path, width=Inches(7))

# Save report
doc_path = "Hafiza_2025_Report_With_Temperature.docx"
doc.save(doc_path)
print(f"Report saved to: {doc_path}")


KeyError: 'Time'

In [1]:
# EVC Data Visualization - Daily & Monthly Graphs

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# Setup
plt.style.use('seaborn-whitegrid')
sns.set_context("notebook")

# Load Data
file_path = "724000005- Hafiza Trading 4557562_07_Day Arc.csv"
df = pd.read_csv(file_path)

# Convert Time Column
df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d, %H:%M:%S')
df.set_index('Time', inplace=True)

# Rename Columns
df.rename(columns={
    'Vb [m3]': 'Uncorrected Volume',
    'Vm [m3]': 'Corrected Volume',
    'p.MP  [ bar]': 'Pressure (bar)',
    'T.MP [{C]': 'Temperature (C)',
    'Diffrence': 'Hourly Usage'
}, inplace=True)

# Convert Pressure to psig
df['Pressure (psig)'] = df['Pressure (bar)'] * 14.5038

# Resample Data
daily_df = df.resample('D').agg({
    'Hourly Usage': 'sum',
    'Pressure (psig)': 'mean',
    'Temperature (C)': 'mean'
})

monthly_df = df.resample('M').agg({
    'Hourly Usage': 'sum',
    'Pressure (psig)': 'mean',
    'Temperature (C)': 'mean'
})

# ------------------------------------------
# 🔹 Daily Graphs
# ------------------------------------------
fig, ax1 = plt.subplots(figsize=(14, 5))
daily_df['Hourly Usage'].plot(ax=ax1, color='blue', label='Daily Usage (m³)')
ax1.set_ylabel('Gas Usage (m³)', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()
daily_df['Pressure (psig)'].plot(ax=ax2, color='red', label='Pressure (psig)')
daily_df['Temperature (C)'].plot(ax=ax2, color='green', label='Temperature (°C)', linestyle='--')
ax2.set_ylabel('Pressure / Temperature', color='red')
ax2.tick_params(axis='y', labelcolor='red')

ax1.set_title("Daily Usage, Pressure & Temperature")
fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=3)
plt.tight_layout()
plt.show()

# ------------------------------------------
# 🔹 Monthly Graphs
# ------------------------------------------
fig, ax1 = plt.subplots(figsize=(14, 5))
monthly_df['Hourly Usage'].plot(kind='bar', ax=ax1, color='blue', label='Monthly Usage (m³)')
ax1.set_ylabel('Gas Usage (m³)', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()
monthly_df['Pressure (psig)'].plot(ax=ax2, color='red', marker='o', label='Pressure (psig)')
monthly_df['Temperature (C)'].plot(ax=ax2, color='green', marker='s', linestyle='--', label='Temperature (°C)')
ax2.set_ylabel('Pressure / Temperature', color='red')
ax2.tick_params(axis='y', labelcolor='red')

ax1.set_title("Monthly Usage, Pressure & Temperature")
fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=3)
plt.tight_layout()
plt.show()


OSError: 'seaborn-whitegrid' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)